# Imports

In [2]:
from ultralytics import YOLO
import os
from scipy.io import loadmat
import numpy as np

In [13]:
# load model
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]


results_list = []
# iterate for each image
for image_file in IMAGE_FILES:
    # execute
    results = model(source=image_file, show=False, conf=0.3, save=False)
    results_list.append(results)
del model

# USE ANNOTATION MPII

annotations_data = loadmat("./mpii/annotations/mpii_human_pose.mat")
annolist_test = annotations_data['RELEASE'] ['annolist']
#print("Data Structure Annolist Test", annolist_test) 
# annolist_test  two-dimensional array of structure


# extract groups of people
# here I extract groups of people using the getMultiPersonGroups function from the MPII assessment toolset. This is done by loading the groups from a previously generated file.
data = loadmat('./mpii/eval/groups_v12.mat')
groups = data["groups"]

# Load and use getMultiPersonGroups function
def getMultiPersonGroups(groups, release_data):
    imgidxs_multi_test = []
    rectidxs_multi_test = []
    for group in groups:
        imgidxs = []
        rectidxs = []
        for item in group:
            if len(item) == 0:  # check if item is empthy
                continue
            if len(item[0]) > 0:
                imgidxs.append(item[0][0])
            if len(item[0]) > 1:
                rectidxs.append(item[0][1])
            else:
                rectidxs.append([])
        imgidxs_multi_test.append(imgidxs)
        rectidxs_multi_test.append(rectidxs)
    return imgidxs_multi_test, rectidxs_multi_test


imgidxs_multi_test, rectidxs_multi_test = getMultiPersonGroups(groups, annotations_data['RELEASE'])
########################################################################
print("imgidxs_multi_test", imgidxs_multi_test[:10])
print("rectidxs_multi_test", rectidxs_multi_test[:10])
print("actual structure of imgidxs_multi_test:")
print(type(imgidxs_multi_test))  
#######################################################################


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000001163.jpg: 384x640 2 persons, 579.8ms
Speed: 17.1ms preprocess, 579.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000003072.jpg: 384x640 3 persons, 566.0ms
Speed: 0.0ms preprocess, 566.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000004812.jpg: 384x640 1 person, 643.0ms
Speed: 3.0ms preprocess, 643.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000005283.jpg: 384x640 2 persons, 644.4ms
Speed: 4.0ms preprocess, 644.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\On